### 這堂課目標
<li>Deep Neural Network (DNN)</li>

- <a href="https://drive.google.com/open?id=1io6A-ZRL9KPw1PJ3fjJNij2Pl7RLjOWA">投影片1 : 深度學習類型介紹</a>

- <a href="https://drive.google.com/open?id=1inmvcLopAGhe8h5Zircsb4ToBEgehhxC">投影片2 : 名詞解釋</a>

#### 第0步：安裝套件(keras, tensorflow)
code originally written by Alvira Swalin

In [3]:
## Tensorflow:google deep learning module, keras:high-level, use tensorflow as base
import pandas as pd
import keras
import numpy as np
# import sklearn
# import sklearn.model_selection

ModuleNotFoundError: No module named 'keras'

#### 1.載入資料

In [1]:
df = pd.read_csv('StudentsPerformance.csv')
#data source:https://www.kaggle.com/spscientist/students-performance-in-exams

NameError: name 'pd' is not defined

In [3]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
df.nunique()

gender                          2
race/ethnicity                  5
parental level of education     6
lunch                           2
test preparation course         2
math score                     81
reading score                  72
writing score                  77
dtype: int64

#### 2.清洗資料（data cleaning / Feature engineering）

In [6]:
target = df['math score'] #建立預測目標
one_hot_data = df.drop(['writing score','math score', 'reading score'],axis=1) #準備做one hot encoding
data = pd.get_dummies(one_hot_data) #one hot encoding
data = pd.concat([data,df[['writing score','reading score']]],axis=1) #把數值資料補回去df內

In [8]:
print(data.shape)
data.head()

(1000, 19)


,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none,writing score,reading score
0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,74,72
1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,88,90
2,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,93,95
3,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,44,57
4,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,75,78


#### 3.分訓練集跟測試集（train test split）

In [10]:
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(data,target,
                                                                         test_size=0.2,random_state=10)

## DNN

#### 4.定義模型

In [11]:
model = keras.models.Sequential() #宣告模型種類
model.add(keras.layers.Dense(100, #神經元數量
                             activation='relu', #activation function
                             input_shape=(x_train.shape[1],))) #第一層必須要定義輸入值數量
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(1)) #輸出神經元數量依問題而定

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2000      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 12,201
Trainable params: 12,201
Non-trainable params: 0
_________________________________________________________________


#### 5.訓練模型

In [13]:
model.compile(optimizer='Adam',loss='mean_squared_error')
early_stop = keras.callbacks.EarlyStopping(patience=5)

In [14]:
model.fit(x_train,y_train,validation_split=0.2,epochs=200,callbacks=[early_stop])

Instructions for updating:
Use tf.cast instead.
Train on 640 samples, validate on 160 samples
Epoch 1/200
640/640 [==============================] - 1s 2ms/step - loss: 1623.8217 - val_loss: 89.6808
Epoch 2/200
640/640 [==============================] - 0s 57us/step - loss: 132.1098 - val_loss: 76.6953
Epoch 3/200
640/640 [==============================] - 0s 159us/step - loss: 87.7744 - val_loss: 72.1268
Epoch 4/200
640/640 [==============================] - 0s 70us/step - loss: 79.9191 - val_loss: 69.6303
Epoch 5/200
640/640 [==============================] - 0s 78us/step - loss: 78.3349 - val_loss: 68.6949
Epoch 6/200
640/640 [==============================] - 0s 81us/step - loss: 77.6638 - val_loss: 69.9089
Epoch 7/200
640/640 [==============================] - 0s 94us/step - loss: 76.9167 - val_loss: 67.1488
Epoch 8/200
640/640 [==============================] - 0s 67us/step - loss: 75.1400 - val_loss: 66.1073
Epoch 9/200
640/640 [==============================] - 0s 112us/step - 

#### 6.評估模型表現

In [15]:
dnn_pred = model.predict(x_test)

In [21]:
df = pd.DataFrame(np.concatenate((y_test.values.reshape(-1,1),dnn_pred),axis=1),columns=['gt','dnn_pred'])
df.head(10)

,gt,dnn_pred
0,64.0,60.857822
1,84.0,90.354256
2,64.0,58.446163
3,63.0,63.742512
4,69.0,82.360809
5,72.0,71.716270
6,91.0,95.556488
7,76.0,73.167770
8,40.0,54.346619
9,79.0,80.950623


#### 7.調整參數（重複5,6）

## XGB 比較

In [22]:
import xgboost as xgb
from sklearn import metrics

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))
xgbr = xgb.XGBRegressor(max_depth=50, min_child_weight=1, n_estimators=200,verbosity=1,learning_rate=0.16,)



/Users/harryhowiefish/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/harryhowiefish/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [23]:
eval_set = [(x_test, y_test)]
xgbr.fit(x_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=True,early_stopping_rounds=2)

[0]	validation_0-rmse:58.18891
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:49.29459
[2]	validation_0-rmse:41.75803
[3]	validation_0-rmse:35.47944
[4]	validation_0-rmse:30.22240
[5]	validation_0-rmse:25.77072
[6]	validation_0-rmse:22.11409
[7]	validation_0-rmse:19.02749
[8]	validation_0-rmse:16.38335
[9]	validation_0-rmse:14.25076
[10]	validation_0-rmse:12.53395
[11]	validation_0-rmse:11.10094
[12]	validation_0-rmse:9.95656
[13]	validation_0-rmse:9.03865
[14]	validation_0-rmse:8.44435
[15]	validation_0-rmse:7.83959
[16]	validation_0-rmse:7.44417
[17]	validation_0-rmse:7.07687
[18]	validation_0-rmse:6.86517
[19]	validation_0-rmse:6.69807
[20]	validation_0-rmse:6.56421
[21]	validation_0-rmse:6.45252
[22]	validation_0-rmse:6.39998
[23]	validation_0-rmse:6.32307
[24]	validation_0-rmse:6.26844
[25]	validation_0-rmse:6.22400
[26]	validation_0-rmse:6.21174
[27]	validation_0-rmse:6.19914
[28]	validation_0-rmse:6.17360
[29]	validation_0-rmse:6.16864
[30]

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.16, max_delta_step=0, max_depth=50,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=200, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
       validate_parameters=False, verbosity=1)

In [24]:
xgb_pred = xgbr.predict(x_test)

In [25]:
sklearn.metrics.mean_squared_error(y_test,dnn_pred),sklearn.metrics.mean_squared_error(y_test,xgb_pred)

(32.393575480085055, 37.645149556756316)

In [26]:
df['xgb_pred']=xgb_pred
df.head(10)

,gt,dnn_pred,xgb_pred
0,64.0,60.857822,60.827217
1,84.0,90.354256,89.362518
2,64.0,58.446163,55.751373
3,63.0,63.742512,64.865707
4,69.0,82.360809,81.018883
5,72.0,71.716270,69.969719
6,91.0,95.556488,94.494011
7,76.0,73.167770,73.080208
8,40.0,54.346619,52.371387
9,79.0,80.950623,79.517570


## ML (Machine Learning v.s. NN (Neural Network)
- ML類別資料有比DL有更好的表現
- ML訓練時間比較短
- ML更能表達運算過程的含義（DL常被形容為黑盒子）
- NN可以處理更多更酷的問題

# 無作業